In [7]:
import pandas as pd
from megadetector.visualization import visualization_utils as vis_utils
from megadetector.detection import run_detector

In [16]:
model = run_detector.load_detector('MDV5A')

expected = pd.read_csv('../y_clean.csv', index_col=0)
expected = expected[expected.species != 'human']  # remove humans
expected['image'] = '../../pictures/' + \
    expected.RelativePath.str.replace('\\', '/') + \
    '/' + expected.File
expected = expected[['image', 'species']]
expected

Model v5a.0.1 already exists and is valid at /tmp/megadetector_models/md_v5a.0.1.pt
PyTorch reports 1 available CUDA devices
GPU available: True
Bypassing imports for model type yolov5
Loading PT detector with compatibility mode classic
Loaded image size 1280 from model metadata
Using model stride: 64
PTDetector using device cuda:0


Fusing layers... 
Fusing layers... 
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs


,image,species
0,../../pictures/01_CZARNE/B/Lato/1/2023-07-16 0...,bird
1,../../pictures/01_CZARNE/B/Lato/1/2023-07-21 0...,bird
2,../../pictures/01_CZARNE/B/Lato/1/2023-07-25 1...,bird
3,../../pictures/01_CZARNE/B/Lato/1/2023-07-28 1...,bird
4,../../pictures/01_CZARNE/B/Lato/1/2023-07-29 0...,bird
...,...,...
33296,../../pictures/30_SUCHEDNIÓW/R/Zima/3/2023-02-...,fox
33297,../../pictures/30_SUCHEDNIÓW/R/Zima/3/2023-02-...,roedeer
33298,../../pictures/30_SUCHEDNIÓW/R/Zima/3/2023-03-...,fox
33299,../../pictures/30_SUCHEDNIÓW/R/Zima/3/2023-03-...,marten


#### Single image

In [ ]:
path = '../../pictures/02_WYSZOWATKA/B/Lato/2/'
file = '2023-07-22 19-22-33.JPG'
image_path = path + file

image = vis_utils.load_image(image_path)
result = model.generate_detections_one_image(image)
max(result['detections'], key=lambda d: d.get('conf', float('-inf')))
# if less than 0.2 - not

#### Loop images

In [19]:
image_paths = []
detections = []

for image_path in expected.image:
    image = vis_utils.load_image(image_path)
    result = model.generate_detections_one_image(image)
    conf = max(result['detections'], key=lambda d: d.get('conf', float('-inf')))['conf']

    image_paths.append(image_path)
    detections.append(True if conf > 0.2 else False)

results = pd.DataFrame({'image': image_paths, 'detected': detections})

KeyboardInterrupt: 

In [21]:
len(image_paths)

40

Naah, too long

#### Comparison

,File,RelativePath,species
22,2023-07-25 18-36-14.JPG,01_CZARNE\B\Lato\2,Empty
59,2022-11-28 07-13-27.JPG,01_CZARNE\B\Późna jesień\1,Empty
236,2023-05-11 07-37-33.JPG,01_CZARNE\B\Wiosna\3,Empty
672,2022-10-08 21-36-41.JPG,01_CZARNE\R\Wczesna jesień\2,Empty
730,2023-05-12 04-39-54.JPG,01_CZARNE\R\Wiosna\1,Empty
...,...,...,...
32936,2023-06-01 04-51-53.JPG,30_SUCHEDNIÓW\B\Wiosna\1,Empty
32949,2023-06-07 13-54-48.JPG,30_SUCHEDNIÓW\B\Wiosna\1,Empty
32981,2023-04-19 20-11-44.JPG,30_SUCHEDNIÓW\B\Wiosna\2,Empty
33209,2023-05-15 15-19-07.JPG,30_SUCHEDNIÓW\R\Wiosna\2,Empty


In [10]:
merged = pd.merge(expected, results, on='image', how='left')

KeyError: 'image'